In [1]:
import sys
sys.path.append('scripts/dialent/task1/')
sys.path.append('scripts/dialent/')
sys.path.append('scripts/')
import util

In [2]:
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

In [4]:
u_dev = util.loadAllStandard('./devset/')
u_test = util.loadAllStandard('./testset//')

Failed to load the standard of book_3954:
Unknown mention tag: Facility


In [5]:
def ne_bin(x):
    if x == 'LOC':
        return 0
    elif x == 'LOCORG':
        return 1
    return 2

In [6]:
def get_df(u):
    df = pd.DataFrame(columns=['tokens', 'tokens_id', 'start', 'end', 'ne'])
    for i in range(len(u)):
        res = u[i].makeTokenSets()
        for j in range(len(res)):
            token = res[j].toInlineString()
            split = token.split()
            df = df.append({'tokens': split[4:], 
                            'first_token': str(split[4].split('"')[1]), 
                            'tokens_id': int(split[1]),
                            'ne_bin': ne_bin(str(split[0]))}, ignore_index=True)
    return df

In [12]:
df_dev = get_df(u_dev)
df_dev = df_dev[df_dev['ne_bin'] < 2]

In [13]:
df_test = get_df(u_test)
df_test = df_test[df_test['ne_bin'] < 2]

In [14]:
n = df_dev.shape[0]
vec = TfidfVectorizer(ngram_range=(1,5), analyzer='char',
               min_df=2, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(df_dev['first_token'])
test_term_doc = vec.transform(df_test['first_token'])

In [15]:
Y_dev = df_dev['ne_bin']
Y_test = df_test['ne_bin']

In [16]:
best_parameters = {
    'colsample_bytree': 0.4,
    'gamma': 1.1,
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_weight': 1,
    'reg_alpha': 0
}

In [17]:
clf = xgb.XGBClassifier(**best_parameters)

In [18]:
clf.fit(trn_term_doc, Y_dev)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=1.1, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [19]:
predict = clf.predict(test_term_doc)

In [20]:
u = util.loadAllStandard('./testset/')
for i in range(len(u)):
    res = u[i].makeTokenSets()
    row = []
    for j in range(len(res)):
        token = res[j].toInlineString()
        if token[0] == 'L':
            #Заменить в этом ифе код на код предсказывания
            split = token.split()
            if predict[df_test['tokens_id'] == int(split[1])][0] == 0:
                tag = 'LOC'
            else:
                tag = 'LOCORG'
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (tag, start, end-start+1))
        else:
            split = token.split()
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (split[0], start, end-start+1))
    with open('./result/' + u[i].name + '.task1', 'w') as f:
        f.writelines(row)        

Failed to load the standard of book_3954:
Unknown mention tag: Facility


In [21]:
!python scripts/t1_eval.py -s ./testset -t ./result

Failed to load the standard of book_3954:
Unknown mention tag: Facility
Type    P        R        F1       TP1      TP2      In Std.  In Test.
per        0.9993   0.9993   0.9993  1342.00  1342.00     1343     1343
loc        0.7146   0.6848   0.6994   409.49   409.49      598      573
org        0.9895   0.9895   0.9895  1557.55  1557.55     1574     1574
locorg     0.6714   0.8136   0.7357   515.00   515.00      633      767
overall    0.8981   0.9218   0.9098  3817.04  3817.04     4141     4250
